<span type="title">Action, EL, JSTL and Taglib</span> | <span type="update">2018-09-23</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章在标准 Servlet 和 JSP 规范的基础上介绍简化开发流程的用于 Beans 装卸和常用命令（包含、派发等）的标准动作、用于获取映射和 Beans 数据并支持简单算术运算的 Expression Language 表达式语言、Java Stand Taglib Language 以及 自定义 Taglib 和 xld 来实现函数调用的方法。</p></span>

# 1. 标准动作

标准动作指的是在 JSP 文件中，那些 `<jsp:xxx>` 包裹着的代码。这很像（或者说压根就是） XML 的文件协议。其中 xxx 部分规定了一些可供使用的标签，其中最有用的时： useBean, getProperty, setProperty。很显然，这里的目的是用于在 JSP 文件中构造以及卸载Bean。

## 1.1 使用 Beans

对于卸载Bean（使用Bean）而言，常见的流程是： 用户请求 -> 找到对应的 Servlet -> Servlet 将 Request 的属性中添加一个名为 id，类型为 class，但是转型为 type 的 Java Beans。-> Servlet 分发请求到 JSP 文件， JSP 文件获取并显示 Beans 的某个属性 property。

```java
req.setAttribute("user",new Student("Corkine",22,null));
req.getRequestDispatcher("/hello.jsp").forward(req,resp);
```

JSP 文件中使用 Beans 很简单，通过脚本 `request.getAttritube("id").getProperty()` 即可。但是，最佳实践是，不使用脚本语言，因为脚本往往带来难以维护的代码。使用标准动作可以完成这一调用和使用：

```jsp
<jsp:useBean id="user" class="com.mazhangjing.model.Student" type="com.mazhangjing.model.Person" scope="request" />

<jsp:setProperty name="user" property="address" value="Central China Normal University"/>
<h1>Hello, <jsp:getProperty name="user" property="name" /></h1>
<p>Welcome to my Website. You came from <jsp:getProperty name="user" property="address" /></p>
```

如上所示，useBean 动作从 request 请求域中使用 id = user 获取了类型为 Student 的对象，然后将其转型为 Person，相当于 `Person user = （Student）request.getAttritube("user")`。getProperty 动作使用这个变量名称作为 name，从中获取 name 这个属性，相当于 `user.getName()` 这个调用。

下面介绍一下一些注意事项：

**作用域**： 对于 useBean 而言，如果 scope 字段省略，则默认为 page 作用域。一共有 application、request、session、page 四个作用域。如果 type 字段省略，那么将会用 class 字段的类型来设置引用的类型。如果 class 字段省略，那么将会使用 type 的类型来获取对象并且转型且使用此类型作为引用。不能同时省略 class 和 type。

**引用和实例的类型**： 对于 useBean，还有一个有趣的细节，如果它从给定范围找不到这个对象，那么它会自己创建一个对象，并且放置在 page 作用域中。这对于下一节JSP直接处理请求的情况下较为常用。尤其需要注意，对于创建对象而言，如果不指定 class，而只有 type，则不会自动创建对象。**即 type 只用于转型和定义引用类型， class 用于创建不存在的Beans。因为获取对象不需要知道其本身的类型**，因此对于使用Bean，而不是创建，没有class，只有type也可以工作。

**属性存取动作的注意事项** ：name 指的是我们在本地定义的引用， property 是我们想要获取的属性，value 是我们希望赋给的值。此外，还可以有 param 字段，在下一节介绍。

**对于Java Beans的要求**: 使用标准动作，对于 Java Beans 有要求：必须符合 Beans 的 Property 命名规则，比如 name 属性必须有 getName 和 setName 两个公开可访问的方法。这个 name 可以是 private 的。此外，**必须有无参 public 的构造器**。其次，Beans 的类不能是 abstract 或者 interface，并且可公开访问。

## 1.2 创建 Beans 

对于下列情况：直接从请求到达 jsp 文件，而不经过 servlet，那么，我们就要在 JSP 中构建 Beans：

```html
<form action="/helloagain" method="post">
    Name: <input type="text" name="name" /> <br/>
    School: <input type="text" name="school" /> <br/>
    Age: <input type="text" name="age" /> <br/><br/><br/>
    <input type="submit" />
</form>
```

在 `/helloagain` URL 映射下的 helloagain.jsp 文件代码片段：

```jsp
<jsp:useBean id="user" class="com.mazhangjing.model.Student" type="com.mazhangjing.model.Person">
    <jsp:setProperty name="student" property="name" value="<%= request.getParameter(\"name\") %>"/>
    <jsp:setProperty name="student" property="address" param="school" />
    <jsp:setProperty name="student" property="age" />
    <jsp:setProperty name="student" property="*"/>
</jsp:useBean>

<h1>Hello, <jsp:getProperty name="student" property="name"/>.</h1>
<p>You come from <jsp:getProperty name="student" property="address"/>.
And you are <jsp:getProperty name="student" property="age" /> now.</p>
```

可以看到，区别在于，useBean 因为需要创建对象，所以 class 字段是不可少的，type 字段可以指定或者不指定，这取决于你是否想要转型以及使用更为抽象类型的引用。如果不写 scope，那么默认是在 pageScope。

**setProperty 详谈：** 对于 setProperty 而言，之前我们说过，常用的是 name、property 以及 value 这三个字段，对于 value，我们可以使用脚本从 param 中获得，但是这样很丑，双引号需要转义（这也说明了脚本先处理，然后才是标准动作）。但是，我们可以直接指定 param 字段，从 param 中获取值。如果 param 和 property 是一致的，那么甚至不用写 param，程序会自动处理。此外，如果所有的 property 和 param 都是一致的（proerpty >= param），那么使用星号可以直接建立两者映射。

**Java Beans 值类型的转换问题：** String和基本格式可以按照Bean的需求来转换，比如Bean需要int，那么setProperty会自动转换值为int，然后进行设置。但是，如果value掺杂了脚本，比如name字段那样，String不会自动转换成为int。对于非这些类型，则会强制使用toString得到的结果作为参数传递。

如果需要对于参数进行检查，或者进行 parseInt 之类的东西，最好使用 Servlet 进行处理，然后 forward 到 JSP 页面。JSP页面应该只用于View层，不参与逻辑计算。（当然，说这句话的意思是很多人为了不使用Servlet，直接在JSP上面写一段脚本进行参数检查，这样唯一的优点是jsp的默认映射不用写。而按照标准写法，jsp映射应当独立定义，如果按照标注的话，还不如映射到servlet，然后再转jsp结构清晰，容易修改。）